# Guia do Projeto - Tech Challenge Fase 4 - Parte 01

## Processamento de Vídeo: Fundamentos Teóricos e Práticos

Bem-vindo à primeira etapa do nosso pipeline de análise de vídeo. Antes de aplicarmos Inteligência Artificial para detectar faces ou emoções, precisamos dominar a matéria-prima do nosso trabalho: o **vídeo digital**.

### O que é um Vídeo Digital?

Teoricamente, um vídeo nada mais é do que uma sequência de imagens estáticas (chamadas de **frames** ou quadros) exibidas em alta velocidade para criar a ilusão de movimento. 

Para processar um vídeo computacionalmente, precisamos entender três conceitos fundamentais:

1.  **Frame Rate (FPS - Frames Per Second):** A frequência com que as imagens são exibidas. Vídeos comuns têm 24, 30 ou 60 FPS. Quanto maior o FPS, mais fluido é o movimento, mas maior é o custo computacional para processar.
2.  **Resolução:** As dimensões de cada frame (largura x altura em pixels, ex: 1920x1080). Imagens maiores contêm mais detalhes, mas exigem mais memória.
3.  **Codecs e Containers:** O arquivo `.mp4` é um container que guarda o vídeo comprimido por um codec (como H.264). Para ler o vídeo, precisamos "decodificar" esse arquivo frame a frame.

### Objetivos deste Notebook

Neste notebook, vamos construir a base de todo o projeto:
1.  Entender como a biblioteca **OpenCV** manipula vídeos.
2.  Implementar o padrão de projeto **Generator** para processar vídeos longos sem estourar a memória RAM.
3.  Criar a classe `VideoProcessor`, que será reutilizada em todas as etapas futuras.


### 1. Configuração do Ambiente

Utilizaremos o **OpenCV** (`cv2`), a biblioteca padrão da indústria para Visão Computacional. Ela é escrita em C/C++ e possui wrappers para Python, garantindo altíssima performance.

In [1]:
import cv2
import os
import sys
from typing import Generator, Tuple
import logging

# Configuração de logging para monitorar a execução
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### 2. A Classe VideoProcessor

Para manter nosso código organizado e profissional, não usaremos scripts soltos. Encapsularemos a lógica de leitura de vídeo em uma classe.

#### O Padrão Generator (Gerador)

Um conceito crucial aqui é o uso de **Generators** do Python (a palavra-chave `yield`). 

**Por que usar?**
Imagine um vídeo de 1 hora em Full HD. Se tentássemos carregar todos os frames em uma lista na memória RAM, o computador travaria (são gigabytes de dados). 

O `Generator` permite carregar **um frame de cada vez**, processá-lo e descartá-lo antes de carregar o próximo. Isso torna nosso pipeline extremamente eficiente e capaz de processar vídeos de qualquer tamanho.

In [2]:
class VideoProcessor:
    """
    Classe responsável pelo processamento sequencial de vídeos.
    
    Atua como uma fachada para a complexidade do OpenCV, gerenciando
    a abertura, leitura frame a frame e liberação de recursos.
    """

    def __init__(self, video_path: str):
        """
        Inicializa o processador.
        
        Args:
            video_path (str): Caminho absoluto ou relativo para o arquivo de vídeo.
        """
        # Validação básica de existência do arquivo
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Arquivo de vídeo não encontrado: {video_path}")
        
        self.video_path = video_path
        
        # cv2.VideoCapture é a classe do OpenCV que decodifica o vídeo
        self.cap = cv2.VideoCapture(video_path)

        if not self.cap.isOpened():
            raise ValueError(f"Não foi possível abrir o vídeo: {video_path}")
        
        # Extração de metadados técnicos do vídeo
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        logger.info(
            f"Vídeo carregado com sucesso: {self.width}x{self.height} @ "
            f"{self.fps:.2f}fps, {self.frame_count} frames totais."
        )

    def get_frames(self) -> Generator[Tuple[int, float, cv2.Mat], None, None]:
        """
        Gera frames do vídeo sob demanda (Lazy Loading).
        
        Este método transforma a classe em um iterável eficiente.
        
        Yields:
            Tuple[int, float, cv2.Mat]: Uma tupla contendo:
                - frame_num (int): O índice sequencial do frame (0, 1, 2...)
                - timestamp (float): O tempo exato do frame em segundos.
                - frame (cv2.Mat): A imagem do frame em formato matriz NumPy (BGR).
        """
        frame_num = 0
        
        while True:
            # self.cap.read() decodifica o próximo frame do arquivo
            ret, frame = self.cap.read()
            
            # 'ret' é False quando o vídeo acaba ou ocorre erro
            if not ret:
                break

            # Calculamos o timestamp baseados no FPS
            timestamp = frame_num / self.fps if self.fps > 0 else 0
            
            # A palavra-chave 'yield' pausa a função e entrega o valor,
            # retomando daqui na próxima iteração do loop.
            yield frame_num, timestamp, frame
            
            frame_num += 1

    def release(self):
        """
        Libera o arquivo de vídeo do sistema operacional.
        É uma boa prática fechar recursos externos quando não usados mais.
        """
        self.cap.release()

    def get_video_info(self) -> dict:
        """
        Retorna um dicionário com o resumo técnico do vídeo.
        Útil para relatórios e logs.
        """
        return {
            "fps": self.fps,
            "frame_count": self.frame_count,
            "width": self.width,
            "height": self.height,
            "duration_seconds": (
                self.frame_count / self.fps if self.fps > 0 else 0
            )
        }

### 3. Teste Prático

Vamos instanciar nossa classe e verificar se ela consegue ler corretamente os metadados do vídeo de exemplo `meu_video.mp4`.

In [3]:
video_path = "meu_video.mp4"

try:
    # 1. Instanciação
    processor = VideoProcessor(video_path)
    
    # 2. Obtenção de Metadados
    info = processor.get_video_info()
    
    print("=== Relatório Técnico do Vídeo ===")
    print(f"Resolução: {info['width']}x{info['height']} pixels")
    print(f"Taxa de Quadros (FPS): {info['fps']:.2f}")
    print(f"Total de Frames: {info['frame_count']}")
    print(f"Duração Total: {info['duration_seconds']:.2f} segundos")
    
except Exception as e:
    print(f"Ocorreu um erro crítico: {e}")

INFO:__main__:Vídeo carregado com sucesso: 1280x720 @ 30.00fps, 3326 frames totais.


=== Relatório Técnico do Vídeo ===
Resolução: 1280x720 pixels
Taxa de Quadros (FPS): 30.00
Total de Frames: 3326
Duração Total: 110.87 segundos


### 4. Visualizando a Estrutura dos Dados

Para entender o que o computador "vê", vamos imprimir as propriedades dos primeiros frames. Note como o `shape` (formato da matriz) corresponde à resolução do vídeo (Altura, Largura, Canais de Cor).

In [4]:
print("Iniciando leitura sequencial (Stream)...")

# O loop consome o Generator frame a frame
for frame_num, timestamp, frame in processor.get_frames():
    # Vamos inspecionar apenas os primeiros 3 frames
    if frame_num >= 3:
        break
        
    print(f"\n[Frame {frame_num:04d}]")
    print(f"  Tempo: {timestamp:.3f}s")
    print(f"  Dados (Matriz): {frame.shape} - (Altura, Largura, Canais BGR)")
    print(f"  Tipo de Dado: {frame.dtype} (Inteiros de 8 bits sem sinal)")

# Boa prática: liberar o arquivo ao final
processor.release()
print("\nRecursos liberados.")

Iniciando leitura sequencial (Stream)...

[Frame 0000]
  Tempo: 0.000s
  Dados (Matriz): (720, 1280, 3) - (Altura, Largura, Canais BGR)
  Tipo de Dado: uint8 (Inteiros de 8 bits sem sinal)

[Frame 0001]
  Tempo: 0.033s
  Dados (Matriz): (720, 1280, 3) - (Altura, Largura, Canais BGR)
  Tipo de Dado: uint8 (Inteiros de 8 bits sem sinal)

[Frame 0002]
  Tempo: 0.067s
  Dados (Matriz): (720, 1280, 3) - (Altura, Largura, Canais BGR)
  Tipo de Dado: uint8 (Inteiros de 8 bits sem sinal)

Recursos liberados.


### Conclusão da Parte 01

Neste notebook, estabelecemos a base para o processamento de vídeo do projeto.

**O que foi construído:**
*   **Classe `VideoProcessor`:** Encapsula a complexidade do OpenCV para leitura e gerenciamento de recursos.
*   **Padrão Generator:** Implementação de leitura frame a frame (lazy loading) para eficiência de memória.
*   **Extração de Metadados:** Métodos para obter FPS, resolução e contagem total de frames.

Com o processador de vídeo funcional, o próximo passo é implementar a detecção de faces nos frames extraídos.